### Scraping from DBLP

In [7]:
import requests
import xml.etree.ElementTree as ET
import time

# Define base URL for DBLP API
BASE_URL = 'https://dblp.org/search/publ/api'

# Query parameters
query_params = {
    'q': 'computer science',   # Query for computer science papers
    'format': 'xml',           # Response format
    'h': 100,                  # Number of results per page (max is 100)
    'f': 0                     # Starting index for pagination
}

# Storage for results
all_papers = []
total_records = 1000  # Target number of papers
records_per_request = 100  # Number of records per request

while len(all_papers) < total_records:
    # Make API request
    response = requests.get(BASE_URL, params=query_params)
    response.raise_for_status()

    # Parse XML response
    root = ET.fromstring(response.text)

    # Find and store metadata for each paper
    for hit in root.findall(".//hit"):
        paper_data = {
            "title": hit.find(".//title").text,
            "year": hit.find(".//year").text if hit.find(".//year") is not None else "Unknown",
            "authors": [author.text for author in hit.findall(".//author")],
            "venue": hit.find(".//venue").text if hit.find(".//venue") is not None else "Unknown",
            "url": hit.find(".//ee").text if hit.find(".//ee") is not None else "No URL"
        }
        all_papers.append(paper_data)

    print(f"Retrieved {len(all_papers)} papers so far...")

    # Update for pagination
    query_params['f'] += records_per_request  # Increment starting index for next batch

    # Respect rate limits
    time.sleep(60)  # Add delay to avoid overwhelming the server

# Trim to the number of records
all_papers = all_papers[:total_records]

# Save results to a JSON file
import json
with open('dblp_computer_science_papers.json', 'w') as f:
    json.dump(all_papers, f, indent=2)

print(f"Total papers retrieved and saved: {len(all_papers)}")

Retrieved 100 papers so far...
Retrieved 200 papers so far...
Retrieved 300 papers so far...
Retrieved 400 papers so far...
Retrieved 500 papers so far...
Retrieved 600 papers so far...
Retrieved 700 papers so far...
Retrieved 800 papers so far...
Retrieved 900 papers so far...
Retrieved 1000 papers so far...
Total papers retrieved and saved: 1000
